<a href="https://colab.research.google.com/github/garyjwri/portopt/blob/main/Min_variance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [89]:
mu = np.array([0.06,0.053,0.07,0.08])

In [7]:
s = np.array([[0.0224,0.0184,0.025,0.0288],[0.0184,0.0223,0.0275,0.0299],[0.025,0.0275,0.0403,0.0438],[0.0288,0.0299,0.0438,0.0515]])
s

array([[0.0224, 0.0184, 0.025 , 0.0288],
       [0.0184, 0.0223, 0.0275, 0.0299],
       [0.025 , 0.0275, 0.0403, 0.0438],
       [0.0288, 0.0299, 0.0438, 0.0515]])

In [121]:
w_test=[ 0.49367089 , 0.50632911,0,0]

### Minimum variance using cvxpy

In [62]:
import cvxpy as cp

In [103]:
np.set_printoptions(suppress=True)

In [104]:
w = cp.Variable(4)
constraints = [cp.sum(w) == 1, w >= 0]
variance = cp.quad_form(w, s)
prob = cp.Problem(cp.Minimize(variance), constraints)
prob.solve()
print(w.value)
print(w.value @ s @ w.value)

[ 0.49367089  0.50632911 -0.         -0.        ]
0.020374683544303798


In [105]:
# w = cp.Variable(4)
constraints = [cp.sum(w) == 1, w >= 0, w @ mu >= 0.06575]
# variance = cp.quad_form(w, s)
prob = cp.Problem(cp.Minimize(variance), constraints)
prob.solve()
print(w.value)
print(w.value @ s @ w.value)

[ 0.7125  0.     -0.      0.2875]
0.027427296875000003


### Using scipy.optimize

In [74]:
from scipy.optimize import minimize, LinearConstraint, Bounds

In [132]:
w_test @ s @ w_test

0.0203746835443038

In [131]:
def variance(w,sigma=s):
  return w @ sigma @ w
bounds=Bounds(0,1)
constraint = {'type':'eq','fun': lambda x: np.sum(x)-1}
initial_w = np.ones(4)/4
#initial_w = [1,1,0,0]
res = minimize(variance, initial_w, method='SLSQP', bounds=bounds, constraints=constraint)
print(res.x)
print(res.fun)

[0.51700903 0.48299097 0.         0.        ]
0.02037898643023024


In [120]:
constraints = [{'type':'eq','fun': lambda x: np.sum(x)-1},
               {'type':'ineq','fun': lambda x: x @ mu - 0.06575}]
res = minimize(variance, initial_w, bounds=bounds, constraints=constraints)
print(res.x)
print(res.fun)

[0.7125 0.     0.     0.2875]
0.027427296875000014
